In [1]:
import cobra
import cobra.test

ijo = cobra.test.create_test_model('ecoli')

ijo_con = cobra.manipulation.modify.canonical_form(ijo)
ijo_dual = cobra.design.dual_problem(ijo)
ijo_irrev = ijo.copy()
cobra.manipulation.convert_to_irreversible(ijo_irrev)

In [14]:
ijo_con.optimize()

<Solution 0.98 at 0x7f3401877fd0>

In [16]:
model = ijo_con.copy()
for r in model.reactions:
    met = cobra.Metabolite(r.id + '_upper_bound')
    model.add_metabolites(met)
    met._bound = 1000.
    met._constraint_sense = 'L'
    r.add_metabolites({met: 1.})
model.optimize()

<Solution 0.98 at 0x7f3403543c10>

The typical COBRA optimization problem has the form:

$$
S \cdot \bar{v} = 0
$$

making the reactions in the model irreversible and setting the metabolites to <= constraints allows us to represent the model in the cannonical form
$$
S \cdot \bar{v} \leq \bar{b}, \quad \bar{v} \geq 0  
$$

## Primal (in cannonical form)
### Linear Algebra representation
$$
\begin{align*}
\max_{\bar{v}} \ & c^T \bar{v} \\
\mathrm{s.t} \ & \\
& S^v \cdot \bar{v} \leq \bar{b}\\
& \bar{v} \geq 0  
\end{align*}
$$

### COBRApy representation
$$
\begin{align*}
\max & {\sum_{\forall j \in \mathrm{Reactions}}{(\mathrm{objective\_coefficient_j}} \cdot \mathrm{reaction_j})} \\
\mathrm{s.t} \ & \\
& \sum_{\forall j \in \mathrm{Reactions}}{(\mathrm{coefficient_{ij}} \cdot \mathrm{reaction_j})} \leq \mathrm{metabolite\_bound_i}\\
& \mathrm{reaction_j} \leq \mathrm{upper\_bound_j}\\
& \mathrm{reaction_j} \geq 0
\end{align*}
$$


# OptAux
$$
\begin{align*}
\max_{y_j} & \min_{v_j} v_{uptake}  \\
& \mathrm{s.t} \ & \\
& \sum_{\forall i \in \mathrm{Metabolites}}S_{ij} \cdot v_j = 0, & \forall j \in \mathrm{Reactions}\\
& v_j \geq 0,  & \forall j \in \mathrm{Reactions}  \\
& y_j \cdot \mathrm{LB}_j \leq v_j \leq y_j \cdot \mathrm{UB}_j,  & \forall j \in \mathrm{Reactions}\\
& v_{biomass} = \mathrm{set\_biomass}\\
& v_{uptake} \leq \mathrm{uptake\_threshold}\\
& v_{trace\_uptake} \leq \mathrm{trace\_metabolite\_threshold}\\
y_j = \in (0, 1),  \forall j \in \mathrm{Reactions} \\
\sum_{\forall j \in \mathrm{Reactions}}(1-y_j) \leq \mathrm{number\_knockouts}\\
\end{align*}
$$


In [2]:
print('Number of reactions in irreversible form = %i' % len(ijo_irrev.reactions))
print('Number of reactions in canonical form = %i' % len(ijo_con.reactions))

print('\nNumber of metabolites in irreversible form = %i' % len(ijo_irrev.metabolites))
print('Number of metabolites in canonical form = %i' % len(ijo_con.metabolites))

Number of reactions in irreversible form = 3219
Number of reactions in canonical form = 3219

Number of metabolites in irreversible form = 1805
Number of metabolites in canonical form = 3611


A duplicate constraint (metabolite) is added to the opposite side of the reaction as the original metabolite in the model. This is done so "_constraint_sense" of each metabolite can be set as 'L' (<=) while enforcing that their "_bound" actually is forced to equal zero.

In [3]:
# For reversible reaction
print ijo_con.reactions.GAPD.reaction
print ''
print ijo_con.reactions.GAPD_reverse.reaction
print ''
for met in ijo_con.reactions.GAPD.metabolites:
    print met,  met._bound,  met._constraint_sense

13dpg_c__GE_constraint + g3p_c + h_c__GE_constraint + nad_c + nadh_c__GE_constraint + pi_c --> 13dpg_c + g3p_c__GE_constraint + h_c + nad_c__GE_constraint + nadh_c + pi_c__GE_constraint

13dpg_c + g3p_c__GE_constraint + h_c + nad_c__GE_constraint + nadh_c + pi_c__GE_constraint --> 13dpg_c__GE_constraint + g3p_c + h_c__GE_constraint + nad_c + nadh_c__GE_constraint + pi_c

13dpg_c 0.0 L
h_c 0.0 L
nad_c__GE_constraint -0.0 L
nad_c 0.0 L
13dpg_c__GE_constraint -0.0 L
pi_c__GE_constraint -0.0 L
nadh_c 0.0 L
g3p_c 0.0 L
g3p_c__GE_constraint -0.0 L
nadh_c__GE_constraint -0.0 L
pi_c 0.0 L
h_c__GE_constraint -0.0 L


Extra constraints (metabolites) are added if the lower bound of a reaction is greatear than 0. In this case it is ATPM. This forces that the reaction proceeds at a rate above or equal to the lower bound of the reaction

In [4]:
# For reaction with lower_bound > 0
print ijo_con.reactions.ATPM.reaction, '\n'

for met in ijo_con.reactions.ATPM.metabolites:
    print met, met._bound, met._constraint_sense

ATPM__LB_constraint + adp_c__GE_constraint + atp_c + h2o_c + h_c__GE_constraint + pi_c__GE_constraint --> adp_c + atp_c__GE_constraint + h2o_c__GE_constraint + h_c + pi_c 

h_c 0.0 L
adp_c 0.0 L
ATPM__LB_constraint -3.15 L
pi_c__GE_constraint -0.0 L
atp_c__GE_constraint -0.0 L
atp_c 0.0 L
pi_c 0.0 L
h2o_c 0.0 L
adp_c__GE_constraint -0.0 L
h2o_c__GE_constraint -0.0 L
h_c__GE_constraint -0.0 L


In [9]:
ijo_con.metabolites.query('bound')

[]

----
## Dual (without integer decision variables)
### Linear Algebra representation
$$
\begin{align*}
\min \ & \bar{b}^T \bar{w}\\
\mathrm{s.t} \ & \\
& S^T \cdot \bar{w} \geq \bar{c}\\
& \bar{w} \geq 0  
\end{align*}
$$
### COBRApy representation
$$
\begin{align*}
\min & \sum_{\forall i \in \mathrm{metabolites}}{(\mathrm{metabolite\_bound_i} \cdot \mathrm{dual_i})} + \sum_{ \forall j \in \mathrm{reactions}}{(\mathrm{upper\_bound_j} \cdot \mathrm{dual^{ub}_{j}})} + \sum_{\forall j \in \mathrm{reactions}} (\mathrm{-lower\_bound_j \cdot dual^{lb}_{j} })\\
\mathrm{s.t.} & \\
& \sum_{\forall i \in \mathrm{metabolites}}{(\mathrm{S^T_{ij}} \cdot \mathrm{dual_i})} + \mathrm{dual^{ub}_j} + \mathrm{dual^{lb}_j}\geq \mathrm{objective\_coefficient_j}, \quad \forall j \in \mathrm{reactions} \\
& dual \geq 0
\end{align*}
$$
Note: For most M-model applications, $\mathrm{metabolite\_bound_i} = 0 \ \forall i \in \mathrm{metabolites}$ so this term can be dropped from the formulation. 

A the dual can be created from a model in canonical form using the **dual_problem** function.

This function in words essentially transposes the stoichiometric matrix by:
### 1. Transforming constraints (metabolites) into variables (reactions)
 1. set objective equal to the metabolite._bound. This is zero for most metabolites except the lower bound constraint for ATPM
 2. set lower bound to 0. upper bound to the dual maximum (1000 by default)
 3. Add metabolite stoichiometry as reactions that the metabolite is involved in **with the coefficient equal to -(stoichiometry of metabolite in original reaction**)

In [10]:
ijo_con.reactions.LYXI.reaction

'lyx__L_c + xylu__L_c__GE_constraint --> lyx__L_c__GE_constraint + xylu__L_c'

In [11]:
print ijo_dual.reactions.lyx__L_c__dual.reaction
print ''
print ijo_dual.reactions.lyx__L_c__GE_constraint__dual.reaction

LYXt2pp__dual_constrained_by_c --> LYXI__dual_constrained_by_c

LYXI__dual_constrained_by_c --> LYXt2pp__dual_constrained_by_c


In [12]:
# The only reacion from this part included in the objective is the ATPM
for met in ijo_con.metabolites:
    for r in ijo_dual.reactions.query(met.id):
        if '_upper_bound_constraint' in r.id:
            continue
        if r.objective_coefficient != 0:
            print met, r, r.objective_coefficient

ATPM__LB_constraint ATPM__LB_constraint__dual -3.15


In [13]:
ijo_dual.reactions.GAPD__dual_for_upper_bound_constraint.reaction

'GAPD__dual_constrained_by_c --> '

### 2) Transforming variables (reactions) into constraints (metabolites)
 1. metabolite._constraint_sense = 'L' (<=)
 2. metabolite._bound = - reaction.objective_coefficient (to keep the it as a >= inequality)
 

In [136]:
ijo_dual.metabolites.Ec_biomass_iJO1366_core_53p95M__dual_constrained_by_c.reactions

frozenset({<Reaction Ec_biomass_iJO1366_core_53p95M__dual_for_upper_bound_constraint at 0x7f1e0729da10>,
           <Reaction murein5px4p_p__GE_constraint__dual at 0x7f1e072e8750>,
           <Reaction pe160_p__GE_constraint__dual at 0x7f1e072fa3d0>,
           <Reaction pe161_p__GE_constraint__dual at 0x7f1e072fa410>,
           <Reaction kdo2lipid4_e__GE_constraint__dual at 0x7f1e07341c50>,
           <Reaction zn2_c__GE_constraint__dual at 0x7f1e073821d0>,
           <Reaction ser__L_c__GE_constraint__dual at 0x7f1e073ca210>,
           <Reaction sheme_c__GE_constraint__dual at 0x7f1e073ca310>,
           <Reaction so4_c__GE_constraint__dual at 0x7f1e073ca650>,
           <Reaction thf_c__GE_constraint__dual at 0x7f1e073dd190>,
           <Reaction thmpp_c__GE_constraint__dual at 0x7f1e073dd2d0>,
           <Reaction thr__L_c__GE_constraint__dual at 0x7f1e073dd350>,
           <Reaction trp__L_c__GE_constraint__dual at 0x7f1e073ddd10>,
           <Reaction tyr__L_c__GE_constraint__d

In [135]:
ijo_dual.metabolites.Ec_biomass_iJO1366_core_53p95M__dual_constrained_by_c._bound

-1.0

### 3) Add dual variables (reactions) to account for upper bound of original reactions
 1. set objective equal to original reaction.upper_boudn
 2. set dual reacition.upper_bound to dual maximum (1000 by default)

In [116]:
# The dual vector has dimensions of i + 2*j. 
# One for each metabolite and 2 for each reaction (to account for upper and lower bound constraint)
print ijo_dual.reactions.LYXI__dual_for_upper_bound_constraint.reaction
print ijo_dual.reactions.LYXI__dual_for_upper_bound_constraint.objective_coefficient

LYXI__dual_constrained_by_c --> 
1000.0


In [176]:
ijo_dual2.reactions.Ec_biomass_iJO1366_WT_53p95M__dual_constrained_by_c__dual.objective_coefficient = 1.

In [183]:
ijo_dual2.reactions.EX_glc_e__dual_constrained_by_c__dual.reaction

'glc__D_e__dual__dual_constrained_by_c --> EX_glc_e__dual_for_upper_bound_constraint__dual_constrained_by_c + glc__D_e__GE_constraint__dual__dual_constrained_by_c'

----
## With integer decision variables
### Linear Algebra Representation
$$
\begin{align*}
\max & {(c^T)x} \\
\mathrm{s.t.} & \\
& (A_x)x + (A_y)y \leq b\\
& x \geq 0
\end{align*}
$$
### COBRApy Representation
$$
\begin{align*}
\max & \sum_{\forall j \in \mathrm{reactions}}(\mathrm{objective\_coefficient_j} \cdot \mathrm{reaction_j})\\
\mathrm{s.t.} & \\
& \sum_{\forall j \in \mathrm{reactions}}(\mathrm{coefficient_{ij}} \cdot \mathrm{reaction_j}) +
\sum_{\forall j \in \mathrm{reactions}}(\mathrm{decision\_coeff_{ij}} \cdot \mathrm{decision\_var_j}) \leq \mathrm{metabolite\_bound_i}\\
& \mathrm{reaction_j} \leq \mathrm{upper\_bound_j}\\
& \mathrm{reaction_j} \geq 0\\
& \mathrm{decision\_var_j} = 1 \ \mathrm{or \ decision\_var_j} = 0
\end{align*}
$$
## Dual
### Linear Algebra Representation
$$
\begin{align*}
\min & {(b - (A_y)y)^T} w \\
\mathrm{s.t.} & \\
& (A_x)^T w \geq c\\
& w\geq 0
\end{align*}
$$
This linearlizes (with $y\cdot w \rightarrow z$)to:

$$
\begin{align*}
\min & (b^T)w - {((A_y)^T z \ \mathrm{with}  } \\
\mathrm{s.t.} & \\
& (A_x)^Tw \geq c\\
& w\geq 0 \\
& z  \leq w^{ub} \cdot y \\
& z \leq w \\
& z \geq w - w^{ub}\cdot (1-y)\\
& z \geq 0
\end{align*}
$$
### COBRApy Representation
$$
\begin{align*}
\min & \sum_{\forall i \in \mathrm{metabolites}} \mathrm{metabolite\_bound_i \cdot \mathrm{dual_i}}  + \sum_{\forall j \in \mathrm{reactions}} \mathrm{upper\_bound_j \cdot \mathrm{dual^{ub}_j}}  - \sum_{\forall i, j} \mathrm{decision\_coefficient_{ij} \cdot \mathrm{auxilary\_variable_{ij}}}\\
\mathrm{s.t.} & \\
& - \sum_{\forall i \in \mathrm{metabolites}}(\mathrm{S^T_{ij}} * \mathrm{dual_i}) - \mathrm{dual^{ub}}
              \leq - \mathrm{objective\_coefficient_j}\\
& \mathrm{auxiliary\_var_{ij}} - \mathrm{dual\_maximum\_bound_j} \cdot \mathrm{decision\_var_j}\leq 0\\
& \mathrm{auxiliary\_var_{ij}} - \mathrm{dual_i} \leq 0\\
& - \mathrm{auxiliary\_var_{ij}} + \mathrm{dual_i}  + \mathrm{decision\_var_j} \cdot \mathrm{dual\_maximum\_bound_j} \leq \mathrm{dual\_maximum\_bound_j}\\
& \mathrm{dual\_maximum\_bound_j} \geq \mathrm{dual_i} \geq 0\\
& \mathrm{dual\_maximum\_bound_j} \geq \mathrm{dual^{ub}_j} \geq 0\\
& \mathrm{dual\_maximum\_bound_j} \geq \mathrm{auxilary\_variable_{ij}} \geq 0 \\
& \mathrm{decision\_var_j} = 1 \ \mathrm{or \ decision\_var_j} = 0
\end{align*}
$$

In [146]:
cobra.design.design_algorithms._add_decision_variable(ijo, 'GAPD')

<Reaction GAPD_decision_var at 0x7f1e0c339c90>

In [170]:
ijo.reactions.GAPD_decision_var.x

1.0

In [162]:
ijo.reactions.GAPD.reaction

'GAPD_decision_var_lower_bound + g3p_c + nad_c + pi_c <=> 13dpg_c + GAPD_decision_var_upper_bound + h_c + nadh_c'

In [169]:
ijo.reactions.EX_glc_e.lower_bound = -10
ijo.optimize()

<Solution 20.00 at 0x7f1e10c4b990>

In [153]:
ijo_dual = cobra.design.dual_problem(ijo, 'maximize', ['GAPD_decision_var'])

In [154]:
ijo_dual.reactions.query("GAPD")

[<Reaction GAPD_decision_var_lower_bound__dual at 0x7f1e0d2f3cd0>,
 <Reaction GAPD_decision_var_upper_bound__dual at 0x7f1e0d2f3b50>,
 <Reaction GAPD__dual_for_upper_bound_constraint at 0x7f1e112dd050>,
 <Reaction GAPD_decision_var at 0x7f1e10eabdd0>,
 <Reaction GAPD_reverse__dual_for_upper_bound_constraint at 0x7f1e06be7750>,
 <Reaction GAPD_decision_var__auxiliary__GAPD_decision_var_upper_bound__dual at 0x7f1e10c4b150>,
 <Reaction GAPD_decision_var__auxiliary__GAPD_decision_var_lower_bound__dual at 0x7f1e10c4b250>]

In [155]:
ijo_dual.reactions.GAPD_decision_var.reaction

'1000 GAPD_decision_var__auxiliary__GAPD_decision_var_lower_bound__dual__le_decision + 1000 GAPD_decision_var__auxiliary__GAPD_decision_var_upper_bound__dual__le_decision --> 1000 GAPD_decision_var__auxiliary__GAPD_decision_var_lower_bound__dual__g_dual + 1000 GAPD_decision_var__auxiliary__GAPD_decision_var_upper_bound__dual__g_dual'

In [196]:
ijo_dual.reactions.EX_glc_e_reverse__dual_for_upper_bound_constraint.objective_coefficient

10.0

In [190]:
ijo_dual.metabolites.EX_glc_e_reverse__dual_constrained_by_c.ob

0.0

In [192]:
ijo_con.reactions.EX_glc_e_reverse.upper_bound

10.0

In [202]:
for m, v  in ijo_dual.reactions.GAPD_decision_var__auxiliary__GAPD_decision_var_upper_bound__dual.metabolites.items():
    print m, m._bound, v

GAPD_decision_var__auxiliary__GAPD_decision_var_upper_bound__dual__le_decision 0 1
GAPD_decision_var__auxiliary__GAPD_decision_var_upper_bound__dual__le_dual 0 1
GAPD_decision_var__auxiliary__GAPD_decision_var_upper_bound__dual__g_dual 1000 -1


In [204]:
ijo_dual.reactions.GAPD_decision_var.metabolites

{<Metabolite GAPD_decision_var__auxiliary__GAPD_decision_var_upper_bound__dual__le_decision at 0x7f1e10c4b190>: -1000,
 <Metabolite GAPD_decision_var__auxiliary__GAPD_decision_var_upper_bound__dual__g_dual at 0x7f1e10c4b210>: 1000,
 <Metabolite GAPD_decision_var__auxiliary__GAPD_decision_var_lower_bound__dual__le_decision at 0x7f1e10c4b290>: -1000,
 <Metabolite GAPD_decision_var__auxiliary__GAPD_decision_var_lower_bound__dual__g_dual at 0x7f1e10c4b310>: 1000}

In [206]:
cobra.solvers.

'cglpk'